In [1]:
import os
import ast
import json
import pandas as pd

from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from datetime import datetime

In [2]:
nombre_archivo = 'reviews.csv'

# Carga el archivo CSV en un DataFrame
dfReviews = pd.read_csv(nombre_archivo)

C:\Users\Peluchex\AppData\Local\Temp\ipykernel_7268\3187143262.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  dfReviews = pd.read_csv(nombre_archivo)


In [3]:
dfReviews.head()

,id_restaurante,identificador_plataforma,id_review,id_plataforma,raiting,review,date,useful,funny,cool,id_usuario
0,2,0x88dae191ee505917:0x6ba3e25388d3fad4,895976,Google,5,Excellent Dominican/PuertoRican food. Her Arro...,2016-07-01 20:10:00,NaN,NaN,NaN,111192300499999997952.0
1,2,0x88dae191ee505917:0x6ba3e25388d3fad4,895977,Google,5,The food and service is great . In my top 3 do...,2016-08-17 07:42:00,NaN,NaN,NaN,108990109100000002048.0
2,2,0x88dae191ee505917:0x6ba3e25388d3fad4,895978,Google,5,Amazing food!,2017-05-09 12:34:00,NaN,NaN,NaN,112239077299999997952.0
3,2,0x88dae191ee505917:0x6ba3e25388d3fad4,895979,Google,5,Amazing food!,2017-04-23 17:03:00,NaN,NaN,NaN,114435073999999991808.0
4,2,0x88dae191ee505917:0x6ba3e25388d3fad4,895980,Google,5,Execellent Dominican food!!!,2014-07-05 15:58:00,NaN,NaN,NaN,112048916199999995904.0


In [4]:
dfResultado = dfReviews

# Convertimos la columna formato fecha
dfResultado['date'] = pd.to_datetime(dfResultado['date'])
# De la fecha extraemos el año
dfResultado['nueva_fecha'] = dfResultado['date'].apply(lambda x: x.year)

In [5]:
dfResultado

,id_restaurante,identificador_plataforma,id_review,id_plataforma,raiting,review,date,useful,funny,cool,id_usuario,nueva_fecha
0,2,0x88dae191ee505917:0x6ba3e25388d3fad4,895976,Google,5,Excellent Dominican/PuertoRican food. Her Arro...,2016-07-01 20:10:00,NaN,NaN,NaN,111192300499999997952.0,2016
1,2,0x88dae191ee505917:0x6ba3e25388d3fad4,895977,Google,5,The food and service is great . In my top 3 do...,2016-08-17 07:42:00,NaN,NaN,NaN,108990109100000002048.0,2016
2,2,0x88dae191ee505917:0x6ba3e25388d3fad4,895978,Google,5,Amazing food!,2017-05-09 12:34:00,NaN,NaN,NaN,112239077299999997952.0,2017
3,2,0x88dae191ee505917:0x6ba3e25388d3fad4,895979,Google,5,Amazing food!,2017-04-23 17:03:00,NaN,NaN,NaN,114435073999999991808.0,2017
4,2,0x88dae191ee505917:0x6ba3e25388d3fad4,895980,Google,5,Execellent Dominican food!!!,2014-07-05 15:58:00,NaN,NaN,NaN,112048916199999995904.0,2014
...,...,...,...,...,...,...,...,...,...,...,...,...
1892472,26610,esBGrrmuZzSiECyRBoKvvA,893596,Yelp,2,Very small menu. Pizza is the ONLY food item ...,2021-11-04 02:43:27,0.0,4.0,0.0,8811742672.0,2021
1892473,26610,esBGrrmuZzSiECyRBoKvvA,895093,Yelp,5,This place is great! The space is big with a n...,2021-11-04 13:28:42,1.0,0.0,0.0,8816665680.0,2021
1892474,26610,esBGrrmuZzSiECyRBoKvvA,895446,Yelp,5,So first looking at the menu for this place yo...,2022-01-02 03:35:40,1.0,0.0,0.0,9038627888.0,2022
1892475,26610,esBGrrmuZzSiECyRBoKvvA,895509,Yelp,5,I lived in CT for 10 years and CT style pizza ...,2021-11-26 22:55:22,0.0,0.0,0.0,9217825680.0,2021


In [6]:
def analizar_review(review):
    if isinstance(review, str):
        text_blob = TextBlob(review)
        polarity = text_blob.polarity
        subjectivity = text_blob.subjectivity
        # Analizamos la objetividad del comentario y si es mayor al nivel trabajado la tenemos en cuenta. Si es menor el comentario sera neutro.
        if subjectivity > 0.4:
            if polarity > 0:
                return [1, 0, 0]  # Positiva
            elif polarity < 0:
                return [0, 1, 0]  # Negativa
            else:
                return [0, 0, 1]  # Neutra
        else:
            return [0, 0, 1]  # Neutra
    else:
        return [0, 0, 1]  # Si no es una cadena, la consideramos neutra

# Aplicar la función analizar_review a la columna 'review' y guardar los resultados en nuevas columnas
dfResultado[['positi', 'negati', 'neutro']] = dfResultado['review'].apply(analizar_review).tolist()

In [7]:
dfResultado

,id_restaurante,identificador_plataforma,id_review,id_plataforma,raiting,review,date,useful,funny,cool,id_usuario,nueva_fecha,positi,negati,neutro
0,2,0x88dae191ee505917:0x6ba3e25388d3fad4,895976,Google,5,Excellent Dominican/PuertoRican food. Her Arro...,2016-07-01 20:10:00,NaN,NaN,NaN,111192300499999997952.0,2016,1,0,0
1,2,0x88dae191ee505917:0x6ba3e25388d3fad4,895977,Google,5,The food and service is great . In my top 3 do...,2016-08-17 07:42:00,NaN,NaN,NaN,108990109100000002048.0,2016,1,0,0
2,2,0x88dae191ee505917:0x6ba3e25388d3fad4,895978,Google,5,Amazing food!,2017-05-09 12:34:00,NaN,NaN,NaN,112239077299999997952.0,2017,1,0,0
3,2,0x88dae191ee505917:0x6ba3e25388d3fad4,895979,Google,5,Amazing food!,2017-04-23 17:03:00,NaN,NaN,NaN,114435073999999991808.0,2017,1,0,0
4,2,0x88dae191ee505917:0x6ba3e25388d3fad4,895980,Google,5,Execellent Dominican food!!!,2014-07-05 15:58:00,NaN,NaN,NaN,112048916199999995904.0,2014,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1892472,26610,esBGrrmuZzSiECyRBoKvvA,893596,Yelp,2,Very small menu. Pizza is the ONLY food item ...,2021-11-04 02:43:27,0.0,4.0,0.0,8811742672.0,2021,0,1,0
1892473,26610,esBGrrmuZzSiECyRBoKvvA,895093,Yelp,5,This place is great! The space is big with a n...,2021-11-04 13:28:42,1.0,0.0,0.0,8816665680.0,2021,1,0,0
1892474,26610,esBGrrmuZzSiECyRBoKvvA,895446,Yelp,5,So first looking at the menu for this place yo...,2022-01-02 03:35:40,1.0,0.0,0.0,9038627888.0,2022,1,0,0
1892475,26610,esBGrrmuZzSiECyRBoKvvA,895509,Yelp,5,I lived in CT for 10 years and CT style pizza ...,2021-11-26 22:55:22,0.0,0.0,0.0,9217825680.0,2021,1,0,0


In [8]:
# Eliminamos las columnas que no suman nada al analisis
dfResultado = dfResultado.drop(['date', 'identificador_plataforma', 'id_review', 'id_plataforma', 'review', 'id_usuario' ], axis=1)

In [9]:
dfResultado.head(5)

,id_restaurante,raiting,useful,funny,cool,nueva_fecha,positi,negati,neutro
0,2,5,NaN,NaN,NaN,2016,1,0,0
1,2,5,NaN,NaN,NaN,2016,1,0,0
2,2,5,NaN,NaN,NaN,2017,1,0,0
3,2,5,NaN,NaN,NaN,2017,1,0,0
4,2,5,NaN,NaN,NaN,2014,0,0,1


In [10]:
# Agrupamos las reviews por restaurante y año, sumando las carateristicas de cada review
dfagrupado = dfResultado.groupby(["id_restaurante", "nueva_fecha"]).agg({
    'raiting': 'mean',
    'useful': 'sum',
    'funny': 'sum',
    'cool': 'sum',
    'positi': 'sum',
    'negati': 'sum',
    'neutro': 'sum'
}).reset_index()

In [11]:
dfagrupado

,id_restaurante,nueva_fecha,raiting,useful,funny,cool,positi,negati,neutro
0,2,2014,5.000000,0.0,0.0,0.0,0,0,2
1,2,2016,5.000000,0.0,0.0,0.0,4,0,2
2,2,2017,3.666667,0.0,0.0,0.0,4,0,2
3,4,2021,4.500000,0.0,0.0,0.0,4,0,8
4,6,2021,4.933333,0.0,0.0,0.0,16,0,14
...,...,...,...,...,...,...,...,...,...
106253,26608,2021,3.800000,1.0,1.0,0.0,4,0,1
106254,26609,2020,5.000000,0.0,0.0,0.0,2,0,0
106255,26609,2021,4.500000,3.0,0.0,0.0,4,0,0
106256,26610,2021,4.468750,45.0,14.0,24.0,30,2,0


In [12]:
# Exportamos el analisis
dfagrupado.to_json('reviews_agrupado.json', orient='records', lines=True)